# Zero-Cost Proxy Evaluation  w/ minibatch

Testing zero-cost proxies with zone-agnostic behavior policy model

In [2]:
import numpy as np
import sys
import matplotlib.pyplot as plt
import pandas as pd
import pickle
sys.path.insert(0, "../")
import torch
from ppo import PPO
import glob
from tqdm.notebook import tqdm
import json
from scipy.stats import spearmanr

In [3]:
log_data = pd.read_csv("../data/rule_based_log_data/0_cleaned_log.csv")
with open("../data/rule_based_log_data/action_probs_all_data.pkl", "rb") as f:
    behavior_model = pickle.load(f)
with open("../data/invalid_policy_list.json") as f:
    invalid_policies = json.load(f)["invalid_policies"]

## 30 Day MiniBatch

In [4]:
num_ts_per_day = 4 * 24
num_days = 30
ts_end = num_ts_per_day * num_days
zones = log_data["zone"].unique()

## Loading all Policies

In [5]:
policy_list = sorted(list(glob.glob(f"../policy_library/**.pth")))

### 1. GradNorm

In [5]:
from zero_cost_proxies.grad_norm import GradNorm
policy_scores = {}
for zone in zones:
    print(zone)
    ope_data = log_data[log_data["zone"] == zone].sort_values(by=["timestep"])
    ope_data = log_data[:ts_end]
    for policy in tqdm(policy_list):
        agent = PPO(6, 1, 0.003, 0.0005, 1, 10, 0.2,
                    has_continuous_action_space=True, action_std_init=0.2, 
                    device=torch.device('cpu'), diverse_policies=list(),
                    diverse_weight=0, diverse_increase=True)
        agent.load(policy)
        agent.policy_evaluation = True
        agent.policy_old.set_action_std(0.1)
        gn = GradNorm(agent, behavior_model)
        if policy not in invalid_policies:
            norm = gn.get_grad_norm(ope_data)
        else:
            continue
        if policy not in policy_scores:
            policy_scores[policy] = {}
        if zone not in policy_scores[policy]:
            policy_scores[policy][zone] = norm

Perimeter_top_ZN_3


  0%|          | 0/800 [00:00<?, ?it/s]

/Users/aakashsasikumar/Documents/MSc/Research/Dissimilar Policy/building-MARL/studies/../ope/iw.py:61: RuntimeWarning: divide by zero encountered in log
  return np.log(value / (1- value))


Perimeter_bot_ZN_1


  0%|          | 0/800 [00:00<?, ?it/s]

...

  0%|          | 0/800 [00:00<?, ?it/s]

Perimeter_mid_ZN_1


  0%|          | 0/800 [00:00<?, ?it/s]

Perimeter_bot_ZN_4


  0%|          | 0/800 [00:00<?, ?it/s]

In [8]:
eval_data_loc = "../data/1month_eval.csv"
df = pd.read_csv(eval_data_loc, header=None, names=["datetime","policy","zone","energy"])
spearman_corr = {}
for zone in zones:
    eval_df = df[df["zone"] == zone]

    # invalid_policies = list(set(policy_list) - set(policy_scores.keys()))
    for i_policy in invalid_policies:
        eval_df = eval_df[eval_df["policy"]!=i_policy[3:]]
    eval_df = eval_df.sort_values(by=["energy"])
    score_list = []
    for i, row in eval_df.iterrows():
        score_list.append(policy_scores[f"../{row['policy']}"][zone])

    eval_df["ope_scores"] = score_list
    correlation = spearmanr(eval_df["energy"].values, eval_df["ope_scores"].values)
    spearman_corr[zone] = correlation

In [9]:
spearman_corr

{'Perimeter_top_ZN_3': SpearmanrResult(correlation=-0.5400055593405301, pvalue=1.0491927518966247e-43),
 'Perimeter_bot_ZN_1': SpearmanrResult(correlation=-0.6046943731739386, pvalue=4.9379574811595664e-57),
 'Perimeter_top_ZN_1': SpearmanrResult(correlation=-0.598108763784972, pvalue=1.5821386095232398e-55),
 'Core_bottom': SpearmanrResult(correlation=-0.48372097412774695, pvalue=3.99614460679757e-34),
 'Perimeter_top_ZN_4': SpearmanrResult(correlation=-0.530863728931001, pvalue=5.800638801330856e-42),
 'Core_top': SpearmanrResult(correlation=-0.49917914947774633, pvalue=1.46220614776298e-36),
 'Perimeter_bot_ZN_2': SpearmanrResult(correlation=-0.5333612601728236, pvalue=2.055905267546934e-42),
 'Perimeter_bot_ZN_3': SpearmanrResult(correlation=-0.5466069297877864, pvalue=7.242821468180089e-45),
 'Perimeter_mid_ZN_4': SpearmanrResult(correlation=-0.5242880232347741, pvalue=8.545143698321729e-41),
 'Core_mid': SpearmanrResult(correlation=-0.4984345372225621, pvalue=1.9286499189534247e-

In [11]:
with open("grad_norm_ipw_spearman_corrs_14_06_2022.pkl", "wb+") as f:
    pickle.dump(spearman_corr, f)

### 2. Synflow

In [9]:
from zero_cost_proxies.synflow import SynFlow
policy_scores_synflow = {}
for zone in zones:
    print(zone)
    ope_data = log_data[log_data["zone"] == zone].sort_values(by=["timestep"])
    ope_data = log_data[:ts_end]
    for policy in tqdm(policy_list):
        agent = PPO(6, 1, 0.003, 0.0005, 1, 10, 0.2,
                    has_continuous_action_space=True, action_std_init=0.2, 
                    device=torch.device('cpu'), diverse_policies=list(),
                    diverse_weight=0, diverse_increase=True)
        agent.load(policy)
        agent.policy_evaluation = True
        agent.policy_old.set_action_std(0.1)
        sf = SynFlow(agent)
        if policy not in invalid_policies:
            synflow = sf.get_synflow()
        else:
            continue
        if policy not in policy_scores_synflow:
            policy_scores_synflow[policy] = {}
        if zone not in policy_scores_synflow[policy]:
            policy_scores_synflow[policy][zone] = synflow

Perimeter_top_ZN_3


  0%|          | 0/800 [00:00<?, ?it/s]

Perimeter_bot_ZN_1


  0%|          | 0/800 [00:00<?, ?it/s]

Perimeter_top_ZN_1


...

  0%|          | 0/800 [00:00<?, ?it/s]

Perimeter_mid_ZN_1


  0%|          | 0/800 [00:00<?, ?it/s]

Perimeter_bot_ZN_4


  0%|          | 0/800 [00:00<?, ?it/s]

In [10]:
eval_data_loc = "../data/1month_eval.csv"
df = pd.read_csv(eval_data_loc, header=None, names=["datetime","policy","zone","energy"])
spearman_corr_synflow = {}
for zone in zones:
    eval_df = df[df["zone"] == zone]

    # invalid_policies = list(set(policy_list) - set(policy_scores.keys()))
    for i_policy in invalid_policies:
        eval_df = eval_df[eval_df["policy"]!=i_policy[3:]]
    eval_df = eval_df.sort_values(by=["energy"])
    score_list = []
    for i, row in eval_df.iterrows():
        score_list.append(policy_scores_synflow[f"../{row['policy']}"][zone])

    eval_df["ope_scores"] = score_list
    correlation = spearmanr(eval_df["energy"].values, eval_df["ope_scores"].values)
    spearman_corr_synflow[zone] = correlation

In [11]:
spearman_corr_synflow

{'Perimeter_top_ZN_3': SpearmanrResult(correlation=-0.02399655788801756, pvalue=0.5709350401752371),
 'Perimeter_bot_ZN_1': SpearmanrResult(correlation=-0.007958102162903363, pvalue=0.8510831000493045),
 'Perimeter_top_ZN_1': SpearmanrResult(correlation=0.004610768076634543, pvalue=0.9133850920486921),
 'Core_bottom': SpearmanrResult(correlation=0.0846908842595264, pvalue=0.04534104009312232),
 'Perimeter_top_ZN_4': SpearmanrResult(correlation=0.01057283551657145, pvalue=0.8030336897038484),
 'Core_top': SpearmanrResult(correlation=0.056829866807361164, pvalue=0.17968820761249374),
 'Perimeter_bot_ZN_2': SpearmanrResult(correlation=-0.005958563816778369, pvalue=0.8882133629244287),
 'Perimeter_bot_ZN_3': SpearmanrResult(correlation=0.0023340313100253082, pvalue=0.9560902859952176),
 'Perimeter_mid_ZN_4': SpearmanrResult(correlation=-0.03969206587361125, pvalue=0.34890507616128863),
 'Core_mid': SpearmanrResult(correlation=0.06361473610160781, pvalue=0.13304293548202217),
 'Perimeter_mi

In [12]:
with open("data/synflow_spearman_corr15_06_2022.pkl", "wb+") as f:
    pickle.dump(spearman_corr_synflow, f)

### 3. Jacobian Covariance

In [ ]:
from zero_cost_proxies.jacob_cov import JacobianCovariance
policy_scores_jc = {}
for zone in zones:
    print(zone)
    ope_data = log_data[log_data["zone"] == zone].sort_values(by=["timestep"])
    ope_data = log_data[:ts_end]
    states = []
    for i, row in ope_data.iterrows():
        state_vars = ["outdoor_temp", "solar_irradiation", "time_hour",
                      "zone_humidity", "zone_temp", "zone_occupancy"]
        state = [row[var] for var in state_vars]
        states.append(state)
    states = torch.Tensor(states)
    for policy in tqdm(policy_list):
        agent = PPO(6, 1, 0.003, 0.0005, 1, 10, 0.2,
                    has_continuous_action_space=True, action_std_init=0.2, 
                    device=torch.device('cpu'), diverse_policies=list(),
                    diverse_weight=0, diverse_increase=True)
        agent.load(policy)
        agent.policy_evaluation = True
        agent.policy_old.set_action_std(0.1)
        jc = JacobianCovariance(agent)
        if policy not in invalid_policies:
            synflow = jc.compute_jacob_cov(states)
        else:
            continue
        if policy not in policy_scores_jc:
            policy_scores_jc[policy] = {}
        if zone not in policy_scores_jc[policy]:
            policy_scores_jc[policy][zone] = synflow

Perimeter_top_ZN_3


  0%|          | 0/800 [00:00<?, ?it/s]

Perimeter_bot_ZN_1


  0%|          | 0/800 [00:00<?, ?it/s]

Perimeter_top_ZN_1


...

  0%|          | 0/800 [00:00<?, ?it/s]

Core_top


  0%|          | 0/800 [00:00<?, ?it/s]

Perimeter_bot_ZN_2


  0%|          | 0/800 [00:00<?, ?it/s]

In [17]:
eval_data_loc = "../data/1month_eval.csv"
df = pd.read_csv(eval_data_loc, header=None, names=["datetime","policy","zone","energy"])
spearman_corr_jc = {}
for zone in zones:
    eval_df = df[df["zone"] == zone]

    # invalid_policies = list(set(policy_list) - set(policy_scores.keys()))
    for i_policy in invalid_policies:
        eval_df = eval_df[eval_df["policy"]!=i_policy[3:]]
    eval_df = eval_df.sort_values(by=["energy"])
    score_list = []
    for i, row in eval_df.iterrows():
        score_list.append(policy_scores_jc[f"../{row['policy']}"][zone])

    eval_df["ope_scores"] = score_list
    correlation = spearmanr(eval_df["energy"].values, eval_df["ope_scores"].values)
    spearman_corr_jc[zone] = correlation

In [18]:
spearman_corr_jc

{'Perimeter_top_ZN_3': SpearmanrResult(correlation=-0.0002549434237084198, pvalue=0.9951970973643869)}

In [ ]:
from zero_cost_proxies.snip import SNIP
policy_scores_snip = {}
for zone in zones:
    print(zone)
    ope_data = log_data[log_data["zone"] == zone].sort_values(by=["timestep"])
    ope_data = log_data[:ts_end]
    for policy in tqdm(policy_list):
        agent = PPO(6, 1, 0.003, 0.0005, 1, 10, 0.2,
                    has_continuous_action_space=True, action_std_init=0.2, 
                    device=torch.device('cpu'), diverse_policies=list(),
                    diverse_weight=0, diverse_increase=True)
        agent.load(policy)
        agent.policy_evaluation = True
        agent.policy_old.set_action_std(0.1)
        snip = SNIP(agent, behavior_model)
        if policy not in invalid_policies:
            sn = snip.compute_snip(ope_data)
        else:
            continue
        if policy not in policy_scores_snip:
            policy_scores_snip[policy] = {}
        if zone not in policy_scores_snip[policy]:
            policy_scores_snip[policy][zone] = sn

Perimeter_top_ZN_3


  0%|          | 0/800 [00:00<?, ?it/s]

/Users/aakashsasikumar/Documents/MSc/Research/Dissimilar Policy/building-MARL/studies/../ope/iw.py:61: RuntimeWarning: divide by zero encountered in log
  return np.log(value / (1- value))


Perimeter_bot_ZN_1


  0%|          | 0/800 [00:00<?, ?it/s]

In [ ]:
eval_data_loc = "../data/1month_eval.csv"
df = pd.read_csv(eval_data_loc, header=None, names=["datetime","policy","zone","energy"])
spearman_corr_snip = {}
for zone in zones:
    eval_df = df[df["zone"] == zone]

    # invalid_policies = list(set(policy_list) - set(policy_scores.keys()))
    for i_policy in invalid_policies:
        eval_df = eval_df[eval_df["policy"]!=i_policy[3:]]
    eval_df = eval_df.sort_values(by=["energy"])
    score_list = []
    for i, row in eval_df.iterrows():
        score_list.append(policy_scores_snip[f"../{row['policy']}"][zone])

    eval_df["ope_scores"] = score_list
    correlation = spearmanr(eval_df["energy"].values, eval_df["ope_scores"].values)
    spearman_corr_snip[zone] = correlation

In [ ]:
spearman_corr_snip

In [10]:
with open("data/snip_spearman_corr_16_06_2022.pkl", "wb+") as f:
    pickle.dump(spearman_corr_snip, f)